In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [6]:
!kaggle datasets download -d salader/dogs-vs-cats

 99% 1.06G/1.06G [00:06<00:00, 162MB/s]
100% 1.06G/1.06G [00:06<00:00, 169MB/s]


In [7]:

import zipfile
zip_ref=zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
zip_ref.extractall('/content/')
zip_ref.close()

In [8]:

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [9]:
conv_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(150,150,3)
)

58889256/58889256 [==============================] - 0s 0us/step


In [10]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [11]:
model= Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [13]:
# now we have to freeze the convolutional layer meaning we are not going to train convolutiona layer
conv_base.trainable=False

In [17]:
from tensorflow.keras.utils import load_img, img_to_array,array_to_img

In [18]:
from keras.preprocessing.image import ImageDataGenerator

In [19]:
batch_size=32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
)
test_datagen = ImageDataGenerator(
               rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                  '/content/train',
                  target_size=(150,150),
                  batch_size=batch_size,
                  class_mode='binary'
)
validation_generator = test_datagen.flow_from_directory(
                  '/content/test',
                  target_size=(150,150),
                  batch_size=batch_size,
                  class_mode='binary'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [20]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
# in case of data augmentation we have to train model in this way
history = model.fit_generator(train_generator,epochs=10,validation_data=validation_generator)

<ipython-input-21-f469611baff2>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,epochs=10,validation_data=validation_generator)


Epoch 1/10
625/625 [==============================] - 4271s 7s/step - loss: 0.3173 - accuracy: 0.8648 - val_loss: 0.2165 - val_accuracy: 0.9064
Epoch 2/10
424/625 [===================>..........] - ETA: 17:57 - loss: 0.2445 - accuracy: 0.8962